In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
#Grab all the paths I'll need and print them out along with shape of a dataframe made from them
from glob import glob

for path in glob('competition_data/*.csv'):
    df = pd.read_csv(path)
    print(path,df.shape)

competition_data\bill_of_materials.csv (21198, 17)
competition_data\components.csv (2048, 3)
competition_data\comp_adaptor.csv (25, 20)
competition_data\comp_boss.csv (147, 15)
competition_data\comp_elbow.csv (178, 16)
competition_data\comp_float.csv (16, 7)
competition_data\comp_hfl.csv (6, 9)
competition_data\comp_nut.csv (65, 11)
competition_data\comp_other.csv (1001, 3)
competition_data\comp_sleeve.csv (50, 10)
competition_data\comp_straight.csv (361, 12)
competition_data\comp_tee.csv (4, 14)
competition_data\comp_threaded.csv (194, 32)
competition_data\specs.csv (21198, 11)
competition_data\test_set.csv (30235, 8)
competition_data\train_set.csv (30213, 8)
competition_data\tube.csv (21198, 16)
competition_data\tube_end_form.csv (27, 2)
competition_data\type_component.csv (29, 2)
competition_data\type_connection.csv (14, 2)
competition_data\type_end_form.csv (8, 2)


In [67]:
#establish trainval and test set
tube = pd.read_csv('competition_data/tube.csv')
specs = pd.read_csv('competition_data/specs.csv')
trainval = pd.read_csv('competition_data/train_set.csv')
test = pd.read_csv('competition_data/test_set.csv')

In [18]:
#Get array containing the unique tube assembly ids
trainval_tube_assemblies = trainval['tube_assembly_id'].unique()
test_tube_assemblies =test['tube_assembly_id'].unique()

In [19]:
trainval_tube_assemblies

array(['TA-00002', 'TA-00004', 'TA-00005', ..., 'TA-21195', 'TA-21196',
       'TA-21197'], dtype=object)

In [20]:
from sklearn.model_selection import train_test_split
train_tube_assemblies, val_tube_assemblies = train_test_split(trainval_tube_assemblies, random_state =42)

In [21]:
#Ensure no matching ids
set(train_tube_assemblies)&set(val_tube_assemblies)

set()

In [27]:
#Establish train and val dataframes
train = trainval[trainval['tube_assembly_id'].isin(train_tube_assemblies)]
val = trainval[trainval['tube_assembly_id'].isin(val_tube_assemblies)]
#Ensure shapes equal to trainval shape
train.shape, val.shape, trainval.shape

((22628, 8), (7585, 8), (30213, 8))

In [58]:
bill.isna().sum()

tube_assembly_id        0
component_id_1       2049
quantity_1           2049
component_id_2       6412
quantity_2           6412
component_id_3      16407
quantity_3          16400
component_id_4      20591
quantity_4          20590
component_id_5      21106
quantity_5          21106
component_id_6      21172
quantity_6          21172
component_id_7      21191
quantity_7          21191
component_id_8      21197
quantity_8          21197
dtype: int64

In [63]:
#Separate extra csvs into train and val early
train_tube = tube[tube['tube_assembly_id'].isin(train_tube_assemblies)]
val_tube = tube[tube['tube_assembly_id'].isin(val_tube_assemblies)]

In [68]:
train_specs = specs[specs['tube_assembly_id'].isin(train_tube_assemblies)]
val_specs = specs[specs['tube_assembly_id'].isin(val_tube_assemblies)]

In [61]:
#Add in features from tube.csv
train_w_tubes = train.merge(train_tube, how='left')
val_w_tubes = val.merge(val_tube, how='left')

In [73]:
#The other files are far too small to use for predicting
#Ths two that aren't have proven to contain almost no actual information
#Therefore, realistically, these frames are not likely to actually help
#So I won't be using them in my predictions
useless_train = train_w_tubes.merge(train_specs, how='left')
useless_val = val_w_tubes.merge(val_specs, how='left')
useless_train.shape, useless_val.shape

((22628, 33), (7585, 33))

In [75]:
#Having a look at our features from tube
train_tube.sample(10)

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
17325,TA-17326,SP-0029,25.40,1.65,109.0,4,50.80,N,N,N,N,EF-017,EF-017,0,0,0
12414,TA-12415,SP-0028,6.35,0.89,24.0,2,19.05,N,N,N,N,EF-018,EF-018,0,0,0
7120,TA-07121,SP-0028,50.80,1.65,62.0,2,101.60,N,Y,N,N,EF-017,EF-017,0,0,0
9023,TA-09024,NaN,15.88,1.65,43.0,3,38.10,N,N,N,N,EF-003,EF-003,0,0,0
2675,TA-02676,SP-0029,9.52,1.24,105.0,6,31.75,N,N,N,N,EF-003,EF-003,0,0,0
10750,TA-10751,SP-0029,25.40,1.65,57.0,2,63.50,N,N,N,N,EF-003,EF-003,0,0,0
10469,TA-10470,SP-0028,76.20,1.65,64.0,1,101.60,Y,Y,N,N,EF-012,EF-017,0,0,0
566,TA-00567,SP-0035,6.35,0.89,33.0,2,19.05,N,N,N,N,EF-018,EF-018,0,0,0
10191,TA-10192,SP-0028,63.50,1.65,18.0,0,0.00,N,N,N,N,EF-009,EF-009,0,0,0
5963,TA-05964,SP-0028,38.10,1.65,151.0,2,114.30,N,N,N,N,NONE,NONE,0,0,0


In [81]:
from sklearn.ensemble import RandomForestRegressor
import category_encoders as ce
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(ce.OrdinalEncoder(),
                         RandomForestRegressor(n_estimators = 100, random_state = 42))

In [78]:
def wrangle(X):
    X = X.copy()
    
    # Engineer date features
    X['quote_date'] = pd.to_datetime(X['quote_date'], infer_datetime_format=True)
    X['quote_date_year'] = X['quote_date'].dt.year
    X['quote_date_month'] = X['quote_date'].dt.month
    X = X.drop(columns='quote_date')
    
    # Drop tube_assembly_id because our goal is to predict unknown assemblies
    X = X.drop(columns='tube_assembly_id')
    
    return X

train_wrangled = wrangle(train_w_tubes)
val_wrangled = wrangle(val_w_tubes)

In [82]:
#X features and y target matrices
target = 'cost'
features = train_wrangled.columns.drop(target)
X_train = train_wrangled[features]
X_val = val_wrangled[features]

y_train = train_wrangled[target]
y_val = val_wrangled[target]

In [83]:
#Fit pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['supplier', 'bracket_pricing', 'material_id', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x'],
        drop_invariant=False, handle_missing='value',
        handle_unknown='value',
        mapping=[{'col': 'supplier', 'mapping': S-0066...mators=100, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [85]:
from sklearn.metrics import mean_squared_log_error

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [86]:
y_pred = pipeline.predict(X_val)
rmsle(y_val, y_pred)

0.3476490503671542